### Linea de estampado

Una linea de estampado tiene dos procesos por los cuales atraviesa una pieza. Dependiendo el tipo de estampado los tiempos entre llegada y servicio son los siguientes:

1. prensas manuales
    * tiempo entre llegada (pj: acomodo de pieza de 1 a 2 minutos)
    * tiempo de servicio: 1 minuto
    
2. prensas progresivas
    * tiempo entre llegada (0)
    * tiempo de servicio: 10.5 minutos  

Tiempo de servicio     | 1 | 10.5 |
|------------------------|----|----|
| Probabilidad           | .5 | .5| 
| Probabilidad acumulada |.5| 1|





In [1]:
import random
import numpy as np

num_piezas = 100
tiemposEntreLlegada = []
tiemposServicio = []

def simulacionLlegada():
    tiempo_ell = 0
    pieza = np.random.choice([0,1], p=[0.5,0.5])
    # si es 0 la pieza va a prensa manual
    if(pieza == 0):
        # print('pieza para prensa manual')
        tiempo_ell = evento_acomodo()
        return tiempo_ell
    # 1 la pieza va a prensa progresiva
    # print('pieza para prensa progresiva')
    return tiempo_ell

# Evento cada vez que la pieza va a prensa manual 
def evento_acomodo():
    return random.randint(1,2)


def simulacionServicio(tipo_prensado):
    if(tipo_prensado == 'progresiva'):
        return 10.5
    else:
        return 1
        

# cargar la lista de tiempos de entre llegada de las piezas y tiempos de servicio
for i in range(num_piezas):
    # primer pieza llega en tiempo 0
    if (i == 0):
        tiempo_ell = simulacionLlegada()
        # si es pieza para prensa progresiva el tiempo entre llegada es 0
        if(tiempo_ell == 0):
            tiemposEntreLlegada.append(0)
            tiemposServicio.append(simulacionServicio('progresiva'))
        # la pieza es para prensa manual
        else:
            tiemposEntreLlegada.append(0)
            tiemposServicio.append(simulacionServicio('manual'))
    else:
        tiempo_ell = simulacionLlegada()
        # si es pieza para prensa progresiva el tiempo entre llegada es 0
        if(tiempo_ell == 0):
            tiemposEntreLlegada.append(0)
            tiemposServicio.append(simulacionServicio('progresiva'))
        # la pieza es para prensa manual
        else:
            tiemposEntreLlegada.append(tiempo_ell)
            tiemposServicio.append(simulacionServicio('manual'))


print(f'Numero de piezas: {len(tiemposEntreLlegada)}')
print(f'Tiempos entre llegada: {tiemposEntreLlegada}')
print(f'Tiempos de servicio: {tiemposServicio}')

# print(len(tiemposEntreLlegada))
# print(len(tiemposServicio))

Numero de piezas: 100
Tiempos entre llegada: [0, 0, 0, 2, 2, 0, 1, 0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 2, 0, 1, 1, 0, 2, 2, 1, 2, 0, 0, 2, 0, 0, 1, 0, 0, 1, 1, 1, 1, 2, 0, 0, 1, 1, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 1, 1, 2, 0, 0, 0, 2, 0]
Tiempos de servicio: [1, 10.5, 10.5, 1, 1, 10.5, 1, 10.5, 10.5, 10.5, 10.5, 1, 1, 1, 1, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 1, 10.5, 10.5, 1, 1, 1, 1, 10.5, 1, 1, 10.5, 1, 10.5, 1, 1, 10.5, 1, 1, 1, 1, 10.5, 10.5, 1, 10.5, 10.5, 1, 10.5, 10.5, 1, 1, 1, 1, 1, 10.5, 10.5, 1, 1, 10.5, 1, 1, 10.5, 10.5, 10.5, 10.5, 10.5, 1, 10.5, 10.5, 1, 10.5, 1, 10.5, 1, 10.5, 10.5, 1, 1, 10.5, 10.5, 10.5, 10.5, 1, 10.5, 1, 10.5, 10.5, 10.5, 1, 10.5, 1, 10.5, 1, 1, 1, 10.5, 10.5, 10.5, 1, 10.5]


In [2]:
import pandas as pd
# crear tabla de simulacion
tabla_simulacion = []

pieza = 0
for i in range(len(tiemposEntreLlegada)):
    # inicio de servicio
    if(i == 0):
        # primer pieza llega en tiempo 0
        pieza += 1
        tabla_simulacion.append(
            [pieza,  # numero pieza/cliente
             0,  # tiempo entre llegada
             0,  # tiempo de llegada
             tiemposServicio[i],  # tiempo de servicio
             0,  # inicio de servicio
             0,  # tiempo de espera en la fila
             tiemposServicio[i],  # tiempo de finalizacion de servicio
             tiemposServicio[i],  # tiempo total de pieza/cliente en la prensa/el sistema
             0,  # tiempo de inactividad de prensas/servidor
             ])
    else:
        # obtener datos de la pieza/cliente anterior
        datos_pieza_anterior = tabla_simulacion[i-1]
        # aumentar el contador de pieza/cliente
        pieza += 1

        # tiempo entre llegada
        tiempo_entre_llegada = tiemposEntreLlegada[i]

        # tiempo de llegada
        tiempo_llegada = datos_pieza_anterior[2] + tiempo_entre_llegada

        # tiempo de servicio
        tiempo_servicio = tiemposServicio[i]

        # inicio de servicio
        if(datos_pieza_anterior[6] > tiempo_llegada):
            inicio_servicio = datos_pieza_anterior[6]
        else:
            inicio_servicio = tiempo_llegada

        # tiempo de espera en la fila
        tiempo_espera = inicio_servicio - tiempo_llegada

        # tiempo de finalizacion de servicio
        tiempo_finalizacion_servicio = inicio_servicio + tiempo_servicio

        # tiempo total de pieza/cliente en la prensa/el sistema
        tiempo_total = tiempo_finalizacion_servicio - tiempo_llegada

        # tiempo de inactividad de prensas/servidor
        if(datos_pieza_anterior[6] == tiempo_llegada):
            inactividad_servidor = 0
        else:
            inactividad_servidor = tiempo_llegada - datos_pieza_anterior[6]

        # añadir datos de pieza/cliente ala tabla de simulacion
        tabla_simulacion.append(
            [pieza,  # numero pieza/cliente
             tiempo_entre_llegada,  # tiempo entre llegada
             tiempo_llegada,  # tiempo de llegada
             tiempo_servicio,  # tiempo de servicio
             inicio_servicio,  # inicio de servicio
             tiempo_espera,  # tiempo de espera en la fila
             tiempo_finalizacion_servicio,  # tiempo de finalizacion de servicio
             tiempo_total,  # tiempo total de pieza/cliente en la prensa/el sistema
             inactividad_servidor,  # tiempo de inactividad de prensas/servidor
             ])

# print(f'Tabla simulación {tabla_simulacion}')
# print(len(tabla_simulacion))
columnas = [
    'Pieza',
    'Tiempo entre llegada',
    'Tiempo de llegada',
    'Tiempo de servicio',
    'Inicio de servicio',
    'Tiempo de espera en la fila',
    'Tiempo finalización de servicio',
    'Tiempo total del pieza/cliente en el sistema',
    'Tiempo de inactividad de prensas/servidor']


pd.set_option('display.max_rows', None)
df = pd.DataFrame(tabla_simulacion, columns=columnas)
df


,Pieza,Tiempo entre llegada,Tiempo de llegada,Tiempo de servicio,Inicio de servicio,Tiempo de espera en la fila,Tiempo finalización de servicio,Tiempo total del pieza/cliente en el sistema,Tiempo de inactividad de prensas/servidor
0,1,0,0,1.0,0.0,0.0,1.0,1.0,0.0
1,2,0,0,10.5,1.0,1.0,11.5,11.5,-1.0
2,3,0,0,10.5,11.5,11.5,22.0,22.0,-11.5
3,4,2,2,1.0,22.0,20.0,23.0,21.0,-20.0
4,5,2,4,1.0,23.0,19.0,24.0,20.0,-19.0
5,6,0,4,10.5,24.0,20.0,34.5,30.5,-20.0
6,7,1,5,1.0,34.5,29.5,35.5,30.5,-29.5
7,8,0,5,10.5,35.5,30.5,46.0,41.0,-30.5
8,9,0,5,10.5,46.0,41.0,56.5,51.5,-41.0
9,10,0,5,10.5,56.5,51.5,67.0,62.0,-51.5


1. Tiempo promedio de espera en la fila por cada cliente/pieza.

Average Waiting Time = tiempo total de clientes en la fila / numero total de clientes

In [3]:
tiempo_espera_fila = 0
for pieza in tabla_simulacion:
    tiempo_espera_fila += pieza[5]

average_wt = tiempo_espera_fila / len(tabla_simulacion)
print(f'Average waiting time: {average_wt}')

Average waiting time: 256.75


2. La probabilidad que un cliente tiene que esperar en la fila.

Probabilidad de esperar = numero de clientes que esperan / numero total de clientes

In [4]:
piezas_esperan = 0
for pieza in tabla_simulacion:
    if(pieza[5] > 0):
        piezas_esperan += 1

probabilidad_esperar = piezas_esperan / len(tabla_simulacion)
print(f'Probabilidad de esperar: {probabilidad_esperar}')

Probabilidad de esperar: 0.99


3. Probabilidad de que el servidor este inactivo 

Probabilidad de que el servidor este inactivo = tiempo total de inactividad del servidor / tiempo total de la simulación

In [5]:
tiempo_servidor_inactivo = 0
tiempo_simulacion = 0
for pieza in tabla_simulacion:
    tiempo_servidor_inactivo += pieza[8]
    tiempo_simulacion += pieza[7]

probabilidad_servidor_inactivo = -tiempo_servidor_inactivo / tiempo_simulacion
print(f'Probabilidad de que el servidor este inactivo: {probabilidad_servidor_inactivo}')

Probabilidad de que el servidor este inactivo: 0.9770344578267405


4. Tiempo promedio de servicio

Tiempo promedio de servicio = tiempo total de servicio / numero total de clientes


In [6]:
tiempo_total_servicio = 0
for pieza in tabla_simulacion:
    tiempo_total_servicio += pieza[3]

tiempo_promedio_servicio = tiempo_total_servicio / len(tabla_simulacion)
print(f'Tiempo promedio de servicio: {tiempo_promedio_servicio}')

Tiempo promedio de servicio: 6.035


5. Tiempo promedio de entre llegadas

Tiempo promedio de entre llegadas = suma de todos los tiempos de entre llegadas / (numero de llegadas - 1)

In [7]:
tiempo_entre_llegadas = 0
for pieza in tabla_simulacion:
    tiempo_entre_llegadas += pieza[1]

tiempo_promedio_entre_llegadas = tiempo_entre_llegadas / (len(tabla_simulacion)-1)
print(f'Tiempo promedio de entre llegadas: {tiempo_promedio_entre_llegadas}')

Tiempo promedio de entre llegadas: 0.6666666666666666


6. Promedio del tiempo que pasa un cliente en un sistema

Promedio del tiempo que pasa un cliente en un sistema = total de tiempo que los clientes pasan en el sistema / numero total de clientes

O

Promedio del tiempo que pasa un cliente en un sistema = Promedio del tiempo que pasa un cliente en la cola + Promedio del tiempo que pasa un cliente en servicio

In [8]:
tiempo_sistema = 0
for pieza in tabla_simulacion:
    tiempo_sistema += pieza[7]

promedio_cliente_sistema = tiempo_sistema / len(tabla_simulacion)
print(f'Tiempo prmedio que pasa un cliente en el sistema: {promedio_cliente_sistema}')

Tiempo prmedio que pasa un cliente en el sistema: 262.785
